# Editor

## **Setup**

In [1]:
# ==============================================================================
# CELDA 1: IMPORTACIONES Y DEFINICIÓN DE FUNCIONES (¡VERSIÓN 2.0 ACTUALIZADA!)
# (Ejecutar esta celda una sola vez)
# ==============================================================================

import os
import subprocess
import webvtt
import google.generativeai as genai
import json
import sys
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, CompositeVideoClip, ImageClip
from io import StringIO
import requests
import shutil
from dotenv import load_dotenv

# --- ¡NUEVOS IMPORTS PARA LA SOLUCIÓN ESTABLE DE EMOJIS! ---
from PIL import Image, ImageDraw, ImageFont
import numpy as np
# -------------------------------------------------------------

print("Funciones e importaciones (V2.0 - Estable) listas.")

# --- Cargar API Key ---
load_dotenv()
API_KEY = os.getenv("API_KEY")
if API_KEY:
    genai.configure(api_key=API_KEY)
    print("API Key cargada y configurada.")
else:
    print("ADVERTENCIA: No se encontró API_KEY en el archivo .env", file=sys.stderr)


# --- FUNCIONES DE TIEMPO (Sin Cambios) ---
def to_seconds(time_str):
    try:
        time_str = time_str.split('.')[0]
        parts = list(map(int, time_str.split(':')))
        if len(parts) == 3:  # HH:MM:SS
            h, m, s = parts
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:  # MM:SS
            m, s = parts
            return m * 60 + s
        else:
            raise ValueError("Formato de tiempo no válido")
    except ValueError as e:
        print(f"Error al convertir tiempo '{time_str}': {e}", file=sys.stderr)
        return 0

def from_seconds(total_seconds):
    h = int(total_seconds // 3600)
    m = int((total_seconds % 3600) // 60)
    s = int(total_seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

# --- FUNCIÓN UTILITARIA PARA DESCARGAR FUENTE DE EMOJIS (Sin Cambios) ---
def descargar_fuente_emoji(font_path="NotoColorEmoji.ttf"):
    print(f"Buscando la fuente de emoji local: {font_path}...")
    if os.path.exists(font_path):
        print(f"Fuente de emoji encontrada: {font_path}")
        return font_path
    else:
        print("--- ¡ERROR GRAVE! ---", file=sys.stderr)
        print(f"No se pudo encontrar el archivo de fuente: {font_path}", file=sys.stderr)
        print("Por favor, descarga la fuente manualmente desde:", file=sys.stderr)
        print("https://github.com/googlefonts/noto-emoji/raw/main/fonts/NotoColorEmoji.ttf", file=sys.stderr)
        print("Y guárdala en la misma carpeta que este script.", file=sys.stderr)
        return None

# --- FUNCIÓN DE SUBTÍTULOS (Sin Cambios) ---
def leer_subtitulos_locales(vtt_path):
    captions = None
    transcript = ""
    try:
        print(f"Leyendo archivo de subtítulos local: {vtt_path}...")
        if not os.path.exists(vtt_path):
            raise FileNotFoundError(f"No se pudo encontrar el archivo de subtítulos en: {vtt_path}")
        with open(vtt_path, 'r', encoding='utf-8') as f:
            vtt_content = f.read()
        captions = webvtt.read_buffer(StringIO(vtt_content))
        for caption in captions:
            timestamp = caption.start
            timestamp_clean = timestamp.split('.')[0]
            transcript += f"[{timestamp_clean}] {caption.text.strip().replace(chr(10), ' ')}\n"
        print(f"Subtítulos leídos y formateados exitosamente.")
        return transcript, captions
    except Exception as e:
        print(f"Error leyendo o procesando el archivo VTT: {e}", file=sys.stderr)
        return None, None

# --- FUNCIÓN DE IA (CORTES) (Sin Cambios) ---
def obtener_cortes_para_eliminar(transcripcion, duracion_minima_seg=0.1):
    print(f"Conectando con Gemini para analizar relleno verbal (duración > {duracion_minima_seg}s)...")
    try:
        model = genai.GenerativeModel('gemini-2.0-flash')
        prompt = f"""
        Eres un editor de video cuya tarea es eliminar "relleno" obvio
        de una transcripción.

        Analiza la siguiente transcripción e identifica segmentos para ELIMINAR
        basado en dos criterios. Sé conservador.

        CRITERIOS PARA ELIMINAR:
        1.  **Muletillas / Rellenos:** Palabras o sonidos como 'ehh', 'mmm',
            'pues...', 'esteee...', 'o sea...', 'bueno...'.
            Tu objetivo es eliminar las que *interrumpen la fluidez*.
        2.  **Reinicios Falsos CLAROS:** Oraciones que el orador
            empieza, abandona a mitad de frase, y vuelve a empezar
            (ej. "Y el video... no, mejor dicho, el audio...")

        REGLAS CRÍTICAS:
        -   **NO ELIMINES NADA POR 'REPETICIÓN'.**
        -   **SÉ CONSERVADOR.** Si dudas, NO LO CORTES.
        -   Si el video está limpio, devuelve una lista vacía [].

        Tu respuesta DEBE ser únicamente un objeto JSON:
        - "motivo": ("muletilla", "reinicio falso")
        - "inicio": El timestamp 'HH:MM:SS' de inicio.
        - "fin": El timestamp 'HH:MM:SS' de fin.

        Ejemplo de respuesta:
        [
          {{"motivo": "muletilla", "inicio": "00:02:15", "fin": "00:02:17"}}
        ]

        Aquí está la transcripción:
        ---
        {transcripcion}
        ---
        """
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        print("Análisis de IA completado. Parseando y filtrando segmentos...")
        secciones = json.loads(cleaned_response)
        secciones_filtradas = []
        for s in secciones:
            try:
                inicio_seg = to_seconds(s['inicio'])
                fin_seg = to_seconds(s['fin'])
                duracion_corte = fin_seg - inicio_seg
                if duracion_corte > 0 and duracion_corte >= duracion_minima_seg:
                    secciones_filtradas.append(s)
                else:
                    print(f"  · Descartando corte de IA (duración inválida o muy corta): {s}")
            except Exception:
                pass 
        print(f"Se identificaron {len(secciones_filtradas)} cortes de IA válidos (duración >= {duracion_minima_seg}s).")
        return secciones_filtradas
    except Exception as e:
        print(f"Error al contactar con Gemini o parsear JSON: {e}", file=sys.stderr)
        return []

# --- FUNCIÓN DE IA PARA EMOJIS (Sin Cambios) ---
def obtener_emojis_para_subtitulos(captions_obj, secciones_para_eliminar):
    print("Conectando con Gemini para analizar y sugerir emojis...")
    if not captions_obj:
        return []

    cortes_seg = []
    for corte in secciones_para_eliminar:
        try:
            cortes_seg.append((to_seconds(corte['inicio']), to_seconds(corte['fin'])))
        except Exception:
            continue
            
    captions_buenos = []
    transcript_buena = ""
    for cap in captions_obj:
        es_bueno = True
        for inicio, fin in cortes_seg:
            if cap.start_in_seconds >= inicio and cap.start_in_seconds < fin:
                es_bueno = False
                break
        if es_bueno:
            captions_buenos.append(cap)
            timestamp_clean = cap.start.split('.')[0]
            transcript_buena += f"[{timestamp_clean}] {cap.text.strip().replace(chr(10), ' ')}\n"

    if not transcript_buena:
        print("No se encontró transcripción válida después del filtrado.")
        return []

    try:
        model = genai.GenerativeModel('gemini-2.0-flash')
        prompt = f"""
        Eres un editor de video creativo. Analiza la siguiente transcripción
        e identifica palabras o frases clave que puedan ser realzadas
        con un emoji relevante.

        REGLAS:
        1.  Sé selectivo. No añadas emojis a cada línea, solo a
            conceptos visuales fuertes (ej. idea 💡, dinero 💰,
            rápido 🚀, mundo 🌍, amor ❤️, éxito 🏆).
        2.  El emoji debe aparecer durante la palabra clave.
        3.  El "fin" debe ser 1 o 2 segundos después del "inicio".
        4.  Usa los timestamps originales del texto.

        Tu respuesta DEBE ser únicamente un objeto JSON (una lista de objetos):
        - "emoji": El emoji unicode (ej. "💡").
        - "inicio": El timestamp 'HH:MM:SS' de inicio.
        - "fin": El timestamp 'HH:MM:SS' de fin.

        Ejemplo de respuesta:
        [
          {{"emoji": "💡", "inicio": "00:02:15", "fin": "00:02:17"}},
          {{"emoji": "💰", "inicio": "00:05:31", "fin": "00:05:33"}}
        ]

        Aquí está la transcripción:
        ---
        {transcript_buena}
        ---
        """
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        print("Análisis de emojis completado.")
        return json.loads(cleaned_response)
    except Exception as e:
        print(f"Error al contactar con Gemini para emojis: {e}", file=sys.stderr)
        return []

# --- FUNCIÓN DE SILENCIOS (Sin Cambios) ---
def identificar_silencios_largos(captions_obj, umbral_segundos=2.0):
    print(f"Identificando silencios de más de {umbral_segundos} segundos...")
    if not captions_obj:
        return []
    silencios = []
    sorted_captions = sorted(captions_obj, key=lambda c: c.start_in_seconds)
    for i in range(len(sorted_captions) - 1):
        cap_actual = sorted_captions[i]
        cap_siguiente = sorted_captions[i+1]
        gap = cap_siguiente.start_in_seconds - cap_actual.end_in_seconds
        if gap >= umbral_segundos:
            print(f"  · Silencio largo detectado: {from_seconds(cap_actual.end_in_seconds)} a {from_seconds(cap_siguiente.start_in_seconds)}")
            silencios.append({
                "motivo": "silencio largo",
                "inicio": from_seconds(cap_actual.end_in_seconds),
                "fin": from_seconds(cap_siguiente.start_in_seconds)
            })
    return silencios

# --- FUNCIÓN DE ENSAMBLAJE (CORTES) (Sin Cambios) ---
def ensamblar_video_editado(video_local_path, secciones_para_eliminar, video_final_path):
    video_original_path = "video_original_para_editar.mp4"
    try:
        print(f"Usando video local: {video_local_path}")
        if not os.path.exists(video_local_path):
            raise FileNotFoundError(f"No se pudo encontrar el archivo de video en: {video_local_path}")
        print(f"Copiando video a la ruta de trabajo temporal: {video_original_path}...")
        shutil.copy(video_local_path, video_original_path)
        print("Copia completada.")
        video = VideoFileClip(video_original_path)
        video_duration = video.duration
        if not secciones_para_eliminar:
            print("No se especificaron cortes. El video no será modificado.")
            video.close()
            if os.path.exists(video_original_path) and not os.path.exists(video_final_path):
                os.rename(video_original_path, video_final_path)
            return video_final_path

        cortes_ordenados = sorted(secciones_para_eliminar, key=lambda x: to_seconds(x['inicio']))
        cortes_fusionados = []
        if cortes_ordenados:
            current_merged_corte = cortes_ordenados[0]
            for i in range(1, len(cortes_ordenados)):
                next_corte = cortes_ordenados[i]
                current_fin_seg = to_seconds(current_merged_corte['fin'])
                next_inicio_seg = to_seconds(next_corte['inicio'])
                if next_inicio_seg <= current_fin_seg + 0.1:
                    current_merged_corte['fin'] = from_seconds(max(current_fin_seg, to_seconds(next_corte['fin'])))
                else:
                    cortes_fusionados.append(current_merged_corte)
                    current_merged_corte = next_corte
            cortes_fusionados.append(current_merged_corte)
        print("\n--- Segmentos a ELIMINAR (final, después de fusión) ---")
        print(json.dumps(cortes_fusionados, indent=2, ensure_ascii=False))
        print("--------------------------------------------------\n")

        clips_buenos = []
        current_time_seg = 0.0
        for corte in cortes_fusionados:
            inicio_corte_seg = to_seconds(corte['inicio'])
            fin_corte_seg = to_seconds(corte['fin'])
            if fin_corte_seg > video_duration: fin_corte_seg = video_duration
            if inicio_corte_seg > current_time_seg:
                if (inicio_corte_seg - current_time_seg) > 0.1:
                    print(f"Manteniendo clip: {from_seconds(current_time_seg)} a {from_seconds(inicio_corte_seg)}")
                    clips_buenos.append(video.subclip(current_time_seg, inicio_corte_seg))
            current_time_seg = max(current_time_seg, fin_corte_seg)
        if (video_duration - current_time_seg) > 0.1:
            print(f"Manteniendo clip final: {from_seconds(current_time_seg)} a {from_seconds(video_duration)}")
            clips_buenos.append(video.subclip(current_time_seg, video_duration))

        if not clips_buenos:
            print("Advertencia: La lógica de cortes resultó en un video vacío.")
            video.close()
            return None
        
        print(f"\nEnsamblando video final a partir de {len(clips_buenos)} clips...")
        final_video = concatenate_videoclips(clips_buenos)
        # CORRECCIÓN DE TYPO DE ANTES: libx264
        final_video.write_videofile(video_final_path, codec="libx264", audio_codec="aac", logger=None) 
        final_video.close()
        for clip in clips_buenos: clip.close()
        video.close()
        return video_final_path
    except Exception as e:
        print(f"Ha ocurrido un error al ensamblar el video: {e}", file=sys.stderr)
        return None
    finally:
        if os.path.exists(video_original_path):
            os.remove(video_original_path)

# ==============================================================================
# --- ¡¡¡FUNCIÓN DE COMPOSICIÓN DE EMOJIS (V2.0 - SÚPER ESTABLE)!!! ---
# ==============================================================================
def crear_video_con_emojis(video_base_path, video_final_path, lista_emojis, secciones_para_eliminar):
    """
    Toma el video base cortado y le superpone los emojis.
    ¡ESTA VERSIÓN USA PIL + ImageClip PARA EVITAR ERRORES DE TextClip!
    """
    print(f"Iniciando Etapa 2 (V2.0 - Estable): Superposición de Emojis en {video_base_path}...")
    
    # --- Caché para no renderizar el mismo emoji mil veces ---
    emoji_clip_cache = {}
    
    try:
        # 1. Cargar la fuente de emoji (para Pillow)
        font_path = descargar_fuente_emoji()
        if not font_path:
            raise Exception("La fuente de emojis no se pudo encontrar.")
        
        # Ajusta el tamaño de la fuente de Pillow
        font_size = 100 
        pil_font = ImageFont.truetype(font_path, font_size)
        
        # 2. Cargar video base
        video_base = VideoFileClip(video_base_path)

        # 3. Mapeador de timestamps (sin cambios)
        cortes_procesados = sorted(
            [{'inicio': to_seconds(c['inicio']), 'fin': to_seconds(c['fin'])} for c in secciones_para_eliminar],
            key=lambda x: x['inicio']
        )
        def calcular_nuevo_timestamp(tiempo_original_seg):
            tiempo_a_restar = 0.0
            for corte in cortes_procesados:
                if tiempo_original_seg > corte['fin']:
                    tiempo_a_restar += (corte['fin'] - corte['inicio'])
                elif tiempo_original_seg > corte['inicio']:
                    tiempo_a_restar += (tiempo_original_seg - corte['inicio'])
                    break
            return max(0, tiempo_original_seg - tiempo_a_restar)

        # 4. Crear los clips de IMAGEN para cada emoji
        clips_de_emojis_finales = []
        
        for item in lista_emojis:
            try:
                emoji_char = item['emoji']
                inicio_original = to_seconds(item['inicio'])
                fin_original = to_seconds(item['fin'])
                duracion = fin_original - inicio_original
                if duracion <= 0.1: continue

                nuevo_inicio = calcular_nuevo_timestamp(inicio_original)
                if nuevo_inicio + duracion > video_base.duration:
                    duracion = video_base.duration - nuevo_inicio
                if duracion <= 0.1: continue

                print(f"  · Añadiendo emoji {emoji_char} en {from_seconds(nuevo_inicio)} (Original: {item['inicio']})")

                # --- ¡AQUÍ ESTÁ LA NUEVA LÓGICA! ---
                base_emoji_clip = None
                if emoji_char in emoji_clip_cache:
                    # Usar el clip base de la caché
                    base_emoji_clip = emoji_clip_cache[emoji_char]
                else:
                    # 1. Crear una imagen transparente con Pillow
                    # (Hacemos un canvas un poco más grande que la fuente)
                    canvas_size = int(font_size * 1.5)
                    pil_image = Image.new("RGBA", (canvas_size, canvas_size), (0, 0, 0, 0)) # Transparente
                    draw = ImageDraw.Draw(pil_image)
                    
                    # 2. Dibujar el emoji en la imagen
                    # (Usamos 'anchor="mm"' para centrar el emoji en el canvas)
                    try:
                        # Pillow >= 9.2.0 (preferido)
                        draw.text((canvas_size//2, canvas_size//2), emoji_char, font=pil_font, anchor="mm", embedded_color=True)
                    except TypeError:
                        # Pillow < 9.2.0 (fallback)
                        # (La centrado manual puede no ser perfecto)
                        draw.text((canvas_size//2, canvas_size//2), emoji_char, font=pil_font, embedded_color=True)

                    # 3. Convertir la imagen de Pillow a un clip de MoviePy
                    # Convertimos a array de numpy
                    emoji_array = np.array(pil_image)
                    base_emoji_clip = ImageClip(emoji_array)
                    
                    # 4. Guardar en caché
                    emoji_clip_cache[emoji_char] = base_emoji_clip
                
                # 5. Aplicar duración, posición y efectos al clip
                # (Importante: ¡estamos creando una NUEVA copia del clip cada vez!)
                final_clip = base_emoji_clip.copy().set_position(('center', 0.7), relative=True) \
                                         .set_start(nuevo_inicio) \
                                         .set_duration(duracion) \
                                         .fadein(0.2).fadeout(0.2)

                clips_de_emojis_finales.append(final_clip)
                # --- FIN DE LA NUEVA LÓGICA ---

            except Exception as e:
                # Imprimimos el error pero continuamos con los demás emojis
                print(f"Error procesando emoji '{item}': {e}", file=sys.stderr) 

        if not clips_de_emojis_finales:
            # (Mismo fallback que antes)
            print("No se generaron emojis (o todos fallaron). El video final es el video cortado.")
            video_base.close()
            if os.path.exists(video_base_path) and not os.path.exists(video_final_path):
                os.rename(video_base_path, video_final_path)
            return video_final_path

        # 5. Componer el video final
        print(f"Componiendo video base con {len(clips_de_emojis_finales)} emojis...")
        video_final_con_emojis = CompositeVideoClip([video_base] + clips_de_emojis_finales)
        
        video_final_con_emojis.write_videofile(
            video_final_path,
            codec="libx264",
            audio_codec="aac",
            logger=None
        )
        print(f"\n¡Proceso completado! El video con emojis está en '{video_final_path}'.")

        # Limpieza
        video_final_con_emojis.close()
        video_base.close()
        for clip in clips_de_emojis_finales: clip.close()

        if os.path.exists(video_base_path):
             os.remove(video_base_path)
        return video_final_path
    
    except Exception as e:
        print(f"Ha ocurrido un error al crear el video con emojis: {e}", file=sys.stderr)
        return None

/home/elchristog/video_editor_and_shorts/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Funciones e importaciones (V2.0 - Estable) listas.
API Key cargada y configurada.


## **Configuración (Variables y Rutas)**

In [2]:
# ==============================================================================
# CELDA 2: CONFIGURACIÓN DE RUTAS
# (Define tus archivos de entrada y salida aquí)
# ==============================================================================


# --- Directorios ---
OUTPUT_FOLDER = "edited"
INPUT_FOLDER = "to_edit"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --- Archivos de ENTRADA ---
# (Asegúrate de que existan)
RUTA_VIDEO_LOCAL = os.path.join(INPUT_FOLDER, "video.mp4")
RUTA_SUBTITULOS_LOCAL = os.path.join(INPUT_FOLDER, "subs.vtt")

# --- Archivos de SALIDA (generados por los scripts) ---
# Archivo intermedio del Paso 1
RUTA_VIDEO_CORTADO = os.path.join(OUTPUT_FOLDER, "video_cortado.mp4")
# Archivo de metadatos para comunicar los pasos
METADATA_FILE = os.path.join(OUTPUT_FOLDER, "metadata_cortes.json")
# Archivo final del Paso 2
RUTA_VIDEO_FINAL_EMOJIS = os.path.join(OUTPUT_FOLDER, "video_final_con_emojis.mp4")

# --- Chequeo de API Key ---
if not API_KEY or "TU_API_KEY_AQUI" in API_KEY:
    print("Error: No se encontró la API_KEY.", file=sys.stderr)
    print("Asegúrate de tener un archivo .env con 'API_KEY=tu_clave'", file=sys.stderr)
else:
    print(f"API Key lista para usarse.")

# --- Chequeo de Archivos de Entrada ---
if not os.path.exists(RUTA_VIDEO_LOCAL):
    print(f"Error: El archivo de video no se encuentra en '{RUTA_VIDEO_LOCAL}'", file=sys.stderr)
elif not os.path.exists(RUTA_SUBTITULOS_LOCAL):
    print(f"Error: El archivo de subtítulos no se encuentra en '{RUTA_SUBTITULOS_LOCAL}'", file=sys.stderr)
else:
    print("Archivos de entrada encontrados. ¡Todo listo!")


API Key lista para usarse.
Archivos de entrada encontrados. ¡Todo listo!


## **Cortar el Video**

In [3]:
# ==============================================================================
# CELDA 3: PASO 1 - ANÁLISIS Y CORTE DE VIDEO
# (Produce 'video_cortado.mp4' y 'metadata_cortes.json')
# ==============================================================================
print("--- INICIANDO ETAPA 1: CORTE DE VIDEO ---")

try:
    # ETAPA 0: OBTENER DATOS
    transcripcion, subtitulos_obj = leer_subtitulos_locales(RUTA_SUBTITULOS_LOCAL)
    if not transcripcion or not subtitulos_obj:
        raise Exception("No se pudo obtener la transcripción o los subtítulos del archivo local.")

    # ETAPA 1: LÓGICA DE CORTE
    cortes_ia = obtener_cortes_para_eliminar(transcripcion, duracion_minima_seg=0.1)
    cortes_silencio = identificar_silencios_largos(subtitulos_obj, umbral_segundos=1.5)
    todos_los_cortes = cortes_ia + cortes_silencio

    if not todos_los_cortes:
        print("\nIA y análisis de silencios no encontraron nada que eliminar. ¡Video limpio!")
    else:
        print("\n--- Segmentos totales a eliminar (antes de fusión) ---")
        print(json.dumps(todos_los_cortes, indent=2, ensure_ascii=False))
        print("--------------------------------------------------\n")

    # ETAPA 1.5: ENSAMBLAR VIDEO BASE
    ruta_video_cortado_generado = ensamblar_video_editado(
        RUTA_VIDEO_LOCAL, 
        todos_los_cortes,
        RUTA_VIDEO_CORTADO # Ruta de salida definida en Celda 2
    ) 

    if not ruta_video_cortado_generado:
        raise Exception("Falló la etapa de corte de video.")

    print(f"\nEtapa 1 completada. Video cortado guardado en: {ruta_video_cortado_generado}")

    # --- GUARDAR METADATOS PARA EL PASO 2 ---
    print(f"Guardando metadatos de cortes en {METADATA_FILE}...")
    metadata = {
        "ruta_video_cortado": ruta_video_cortado_generado,
        "todos_los_cortes": todos_los_cortes,
        "ruta_subtitulos_original": RUTA_SUBTITULOS_LOCAL
    }
    with open(METADATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print("\n--- ¡ETAPA 1 FINALIZADA! ---")
    print(f"Video cortado: {ruta_video_cortado_generado}")
    print(f"Metadatos: {METADATA_FILE}")
    print("\nAhora puedes ejecutar la CELDA 4 para agregar emojis.")

except Exception as e:
    print(f"El proceso (Etapa 1) falló: {e}", file=sys.stderr)

--- INICIANDO ETAPA 1: CORTE DE VIDEO ---
Leyendo archivo de subtítulos local: to_edit/subs.vtt...
Subtítulos leídos y formateados exitosamente.
Conectando con Gemini para analizar relleno verbal (duración > 0.1s)...
Análisis de IA completado. Parseando y filtrando segmentos...
  · Descartando corte de IA (duración inválida o muy corta): {'motivo': 'muletilla', 'inicio': '00:02:33', 'fin': '00:02:33'}
  · Descartando corte de IA (duración inválida o muy corta): {'motivo': 'muletilla', 'inicio': '00:03:13', 'fin': '00:03:13'}
  · Descartando corte de IA (duración inválida o muy corta): {'motivo': 'muletilla', 'inicio': '00:04:25', 'fin': '00:04:25'}
  · Descartando corte de IA (duración inválida o muy corta): {'motivo': 'muletilla', 'inicio': '00:12:17', 'fin': '00:12:17'}
Se identificaron 12 cortes de IA válidos (duración >= 0.1s).
Identificando silencios de más de 1.5 segundos...

--- Segmentos totales a eliminar (antes de fusión) ---
[
  {
    "motivo": "muletilla",
    "inicio": "00

## **Agregar Emojis**

In [4]:
# ==============================================================================
# CELDA 4: PASO 2 - AGREGAR EMOJIS
# (Lee 'video_cortado.mp4' y 'metadata_cortes.json', produce 'video_final_con_emojis.mp4')
# ==============================================================================
print("--- INICIANDO ETAPA 2: AGREGAR EMOJIS ---")

try:
    # --- CARGAR METADATOS DEL PASO 1 ---
    print(f"Cargando metadatos desde {METADATA_FILE}...")
    if not os.path.exists(METADATA_FILE):
        print(f"Error: No se encuentra el archivo de metadatos '{METADATA_FILE}'.", file=sys.stderr)
        raise FileNotFoundError("Por favor, ejecuta la CELDA 3 primero.")
        
    with open(METADATA_FILE, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    # Extraer datos del JSON
    ruta_video_cortado_leida = metadata.get("ruta_video_cortado")
    todos_los_cortes_leidos = metadata.get("todos_los_cortes", [])
    ruta_subtitulos_original_leida = metadata.get("ruta_subtitulos_original")

    # Validar que los archivos existan
    if not ruta_video_cortado_leida or not os.path.exists(ruta_video_cortado_leida):
        print(f"Error: El video cortado '{ruta_video_cortado_leida}' no se encuentra.", file=sys.stderr)
        raise FileNotFoundError("Asegúrate de que la CELDA 3 haya terminado correctamente.")
    
    if not ruta_subtitulos_original_leida:
        raise ValueError("El archivo de metadatos no especifica la ruta de los subtítulos.")

    print("Metadatos cargados exitosamente.")

    # ETAPA 0 (Repetida): Necesitamos el objeto de subtítulos de nuevo
    _, subtitulos_obj_leidos = leer_subtitulos_locales(ruta_subtitulos_original_leida)
    if not subtitulos_obj_leidos:
        raise Exception("No se pudo volver a leer el archivo de subtítulos.")

    # ETAPA 2: LÓGICA DE EMOJIS
    lista_emojis = obtener_emojis_para_subtitulos(subtitulos_obj_leidos, todos_los_cortes_leidos)

    if not lista_emojis:
        print("IA no sugirió emojis. El proceso ha finalizado.")
        if os.path.exists(ruta_video_cortado_leida) and not os.path.exists(RUTA_VIDEO_FINAL_EMOJIS):
            os.rename(ruta_video_cortado_leida, RUTA_VIDEO_FINAL_EMOJIS)
            print(f"Video final (solo cortado) guardado como: {RUTA_VIDEO_FINAL_EMOJIS}")
    else:
        print("\n--- Emojis sugeridos por la IA ---")
        print(json.dumps(lista_emojis, indent=2, ensure_ascii=False))
        print("------------------------------------\n")

        # ETAPA 2.5: ENSAMBLAR VIDEO FINAL CON EMOJIS
        crear_video_con_emojis(
            ruta_video_cortado_leida,      # Input: El video ya cortado
            RUTA_VIDEO_FINAL_EMOJIS,       # Output: El video final (de Celda 2)
            lista_emojis,
            todos_los_cortes_leidos
        )
        
        # Limpieza final del archivo de metadatos, ya que el proceso se completó
        if os.path.exists(METADATA_FILE):
            os.remove(METADATA_FILE)
            print(f"Metadatos intermedios ({METADATA_FILE}) eliminados.")

        print("\n--- ¡ETAPA 2 FINALIZADA! ---")
        print(f"Video final con emojis guardado en: {RUTA_VIDEO_FINAL_EMOJIS}")

except Exception as e:
    print(f"El proceso (Etapa 2) falló: {e}", file=sys.stderr)

--- INICIANDO ETAPA 2: AGREGAR EMOJIS ---
Cargando metadatos desde edited/metadata_cortes.json...
Metadatos cargados exitosamente.
Leyendo archivo de subtítulos local: to_edit/subs.vtt...
Subtítulos leídos y formateados exitosamente.
Conectando con Gemini para analizar y sugerir emojis...
Análisis de emojis completado.

--- Emojis sugeridos por la IA ---
[
  {
    "emoji": "✅",
    "inicio": "00:01:11",
    "fin": "00:01:13"
  },
  {
    "emoji": "💰",
    "inicio": "00:05:15",
    "fin": "00:05:17"
  },
  {
    "emoji": "✈️",
    "inicio": "00:07:38",
    "fin": "00:07:40"
  },
  {
    "emoji": "⏰",
    "inicio": "00:09:29",
    "fin": "00:09:31"
  },
  {
    "emoji": "🙏",
    "inicio": "00:12:13",
    "fin": "00:12:15"
  },
  {
    "emoji": "💖",
    "inicio": "00:11:57",
    "fin": "00:11:59"
  }
]
------------------------------------

Iniciando Etapa 2 (V2.0 - Estable): Superposición de Emojis en edited/video_cortado.mp4...
Buscando la fuente de emoji local: NotoColorEmoji.ttf...
Fue

Ha ocurrido un error al crear el video con emojis: invalid pixel size


# Shorts

In [7]:
import os
import subprocess
import webvtt
import google.generativeai as genai
import json
import sys
from moviepy.editor import VideoFileClip
from io import StringIO
import shutil # <-- ¡NUEVO IMPORT!

def to_seconds(time_str):
    """Convierte un string de tiempo 'HH:MM:SS' o 'MM:SS' a segundos."""
    try:
        # Añadido para limpiar timestamps como '00:03:05.123'
        time_str = time_str.split('.')[0]
        parts = list(map(int, time_str.split(':')))
        if len(parts) == 3:  # HH:MM:SS
            h, m, s = parts
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:  # MM:SS
            m, s = parts
            return m * 60 + s
        else:
            raise ValueError("Formato de tiempo no válido")
    except ValueError as e:
        print(f"Error al convertir tiempo '{time_str}': {e}", file=sys.stderr)
        return 0

# --- ¡FUNCIÓN MODIFICADA! ---
def leer_subtitulos_locales(vtt_path):
    """
    Lee un archivo .vtt local y lo devuelve
    como un texto formateado Y como un objeto webvtt.
    """
    captions = None
    transcript = ""

    try:
        print(f"Leyendo archivo de subtítulos local: {vtt_path}...")
        if not os.path.exists(vtt_path):
             raise FileNotFoundError(f"No se pudo encontrar el archivo .vtt local: {vtt_path}")

        # La lógica de parseo del script original es buena, la reutilizamos
        captions = webvtt.read(vtt_path)
        for caption in captions:
            # Usamos la conversión de segundos para consistencia
            secs = int(caption.start_in_seconds)
            h = secs // 3600
            m = (secs % 3600) // 60
            s = secs % 60
            timestamp = f"{h:02}:{m:02}:{s:02}"
            transcript += f"[{timestamp}] {caption.text.strip().replace(chr(10), ' ')}\n"

        print("Subtítulos leídos y formateados exitosamente.")
        return transcript, captions

    except Exception as e:
        print(f"Error extrayendo subtítulos locales: {e}", file=sys.stderr)
        return None, None

def obtener_clips_con_gemini(api_key, transcript):
    """
    Envía la transcripción a Gemini y le pide que identifique
    secciones para shorts, devolviendo un JSON.
    (Función sin cambios)
    """
    print("Conectando con Gemini para analizar momentos clave...")
    try:
        genai.configure(api_key=api_key)

        model = genai.GenerativeModel('gemini-2.0-flash')

        prompt = f"""
        Eres un experto productor de video y creador de contenido viral
        para YouTube Shorts.

        Tu tarea es analizar la siguiente transcripción de un video
        (con marcas de tiempo [HH:MM:SS]) e identificar los 3 a 5
        momentos más impactantes, educativos o "virales" que serían
        perfectos para YouTube Shorts.

        Reglas para los clips:
        1. Cada clip debe tener una duración ideal de 30 a 55 segundos.
        2. Los clips no deben superponerse.
        3. Busca "ganchos" (preguntas, declaraciones impactantes),
           resúmenes de puntos clave o conclusiones claras.
        4. **REGLA CRÍTICA DE COMPLETITUD:** El timestamp de 'fin' DEBE
           corresponder al final de una oración o una idea completa.
           No cortes al orador a mitad de frase. Es PREFERIBLE que el
           clip dure unos segundos mÃ¡s (hasta 59 segundos) para
           asegurar que la idea se concluya, en lugar de cortarlo
           abruptamente.

        Tu respuesta DEBE ser únicamente un objeto JSON, sin ningún
        otro texto, markdown o explicaciÃ³n.

        El formato JSON debe ser una lista de objetos, donde cada
        objeto tiene tres claves:
        - "nombre": Un título corto y pegadizo para el clip (usa guiones bajos, sin espacios).
        - "inicio": El timestamp 'HH:MM:SS' de inicio.
        - "fin": El timestamp 'HH:MM:SS' de fin.

        Ejemplo de respuesta:
        [
          {{"nombre": "Error_Comun_al_Estudiar", "inicio": "00:03:05", "fin": "00:03:52"}},
          {{"nombre": "Reseña_Libro_Saunders", "inicio": "00:08:05", "fin": "00:08:47"}}
        ]

        Aquí está la transcripción:
        ---
        {transcript}
        ---
        """

        response = model.generate_content(prompt)

        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")

        print("Análisis de IA completado. Parseando secciones...")
        secciones = json.loads(cleaned_response)
        return secciones

    except Exception as e:
        print(f"Error al contactar con Gemini o parsear JSON: {e}", file=sys.stderr)
        if "404" in str(e) and "is not found" in str(e):
            print("\n--- ¡PROBLEMA DE API! ---", file=sys.stderr)
            print("Error 404: El modelo no se encuentra.", file=sys.stderr)
            print("Esto casi siempre significa que tu API KEY tiene un problema.", file=sys.stderr)
            print("1. Tu clave de API fue (o será) desactivada por seguridad.", file=sys.stderr)
            print("2. Debes habilitar la API 'Generative Language API' en tu proyecto de Google Cloud.", file=sys.stderr)
        return None

# --- ¡FUNCIÓN MODIFICADA! ---
def cortar_video_clips(video_local_path, secciones, captions_obj):
    """
    Usa el video local, corta las secciones de la IA
    y guarda un .txt con el subtítulo de cada una.
    """
    video_original_path = "video_original.mp4" # Nombre temporal

    try:
        # --- ¡BLOQUE DE DESCARGA ELIMINADO! ---
        # Reemplazado por una copia local
        print(f"Usando video local: {video_local_path}")
        if not os.path.exists(video_local_path):
            raise FileNotFoundError(f"No se pudo encontrar el video en: {video_local_path}")
        
        print(f"Copiando video a la ruta de trabajo temporal: {video_original_path}...")
        shutil.copy(video_local_path, video_original_path)
        print("Copia completada.")
        # --- FIN DE LA MODIFICACIÓN ---

        output_folder = "shorts_generados_por_ia"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        for seccion in secciones:
            nombre = seccion['nombre']
            inicio_str = seccion['inicio']
            fin_str = seccion['fin']

            video = None
            clip_original = None
            clip_recortado = None

            try:
                # Esta parte funciona igual, ya que usa el archivo temporal
                video = VideoFileClip(video_original_path)

                inicio_seg = to_seconds(inicio_str)
                fin_seg = to_seconds(fin_str)

                if fin_seg > video.duration:
                    print(f"Advertencia: Tiempo final '{fin_str}' excede duración. Cortando al final del video.")
                    fin_seg = video.duration

                if inicio_seg >= fin_seg:
                    print(f"Advertencia: Tiempo de inicio '{inicio_str}' es mayor o igual al de fin '{fin_str}'. Saltando clip.")
                    continue

                print(f"Creando clip IA: {nombre} ({inicio_str} a {fin_str})...")

                clip_original = video.subclip(inicio_seg, fin_seg)

                (w, h) = clip_original.size
                target_width = h * 9 / 16
                x_center = w / 2
                x1 = x_center - (target_width / 2)
                x2 = x_center + (target_width / 2)
                clip_recortado = clip_original.crop(x1=x1, y1=0, x2=x2, y2=h)

                output_path = os.path.join(output_folder, f"{nombre}.mp4")
                clip_recortado.write_videofile(output_path,
                                              codec="libx264",
                                              audio_codec="aac",
                                              logger=None)

                print(f"Clip '{nombre}' creado exitosamente.")

                # --- Lógica para guardar .txt (sin cambios) ---
                texto_del_short = []
                if captions_obj:
                    for caption in captions_obj:
                        if caption.start_in_seconds < fin_seg and caption.end_in_seconds > inicio_seg:
                            clean_text = caption.text.strip().replace('\n', ' ')
                            texto_del_short.append(clean_text)

                texto_completo = "\n".join(texto_del_short)
                txt_output_path = os.path.join(output_folder, f"{nombre}.txt")

                try:
                    with open(txt_output_path, 'w', encoding='utf-8') as f:
                        f.write(texto_completo)
                    print(f"Texto del clip '{nombre}' guardado en .txt.")
                except Exception as e:
                    print(f"Advertencia: No se pudo guardar el .txt para '{nombre}': {e}")
                # --- FIN DE LA SECCIÓN ---

            finally:
                if clip_recortado: clip_recortado.close()
                if clip_original: clip_original.close()
                if video: video.close()
        
        # Limpiamos el video temporal copiado
        os.remove(video_original_path)
        print(f"\n¡Proceso completado! Los shorts y .txt están en la carpeta '{output_folder}'.")

    except Exception as e:
        print(f"Ha ocurrido un error al cortar los clips: {e}", file=sys.stderr)
        if os.path.exists(video_original_path):
            os.remove(video_original_path)

# --- Configuración Principal (Orquestador) ---

if __name__ == "__main__":

    # --- ¡EDITA ESTAS LÍNEAS! ---
    # Debes proporcionar la ruta a TUS archivos locales
    RUTA_VIDEO_LOCAL = "to_edit/video.mp4"
    RUTA_SUBTITULOS_LOCAL = "to_edit/subs.vtt"
    # -----------------------------------

    API_KEY = os.getenv("API_KEY")

    # --- ¡CORREGIDO! ---
    # Comprobando la variable "API_KEY" (en lugar de "GEMINI_API_KEY")
    if not API_KEY:
        print("Error: No se encontró la variable de entorno 'API_KEY'.", file=sys.stderr)
        print("Por favor, configúrala antes de ejecutar el script.", file=sys.stderr)
        print(" (Recuerda crear un archivo .env o exportarla en tu terminal)", file=sys.stderr)
        sys.exit(1)

    # --- ¡NUEVO! Comprobación de archivos locales (Corregida) ---

    # Simplemente comprueba si la ruta NO existe.
    if not os.path.exists(RUTA_VIDEO_LOCAL):
        print(f"Error: El archivo de video NO se encuentra en la ruta especificada:", file=sys.stderr)
        print(f"{RUTA_VIDEO_LOCAL}", file=sys.stderr)
        print("Por favor, verifica que la variable RUTA_VIDEO_LOCAL sea correcta.", file=sys.stderr)
        sys.exit(1)

    if not os.path.exists(RUTA_SUBTITULOS_LOCAL):
        print(f"Error: El archivo de subtítulos NO se encuentra en la ruta especificada:", file=sys.stderr)
        print(f"{RUTA_SUBTITULOS_LOCAL}", file=sys.stderr)
        print("Por favor, verifica que la variable RUTA_SUBTITULOS_LOCAL sea correcta.", file=sys.stderr)
        sys.exit(1)
        
    # Si el script llega aquí, es porque los encontró.
    print("¡Éxito! Archivos de video y subtítulos encontrados.")
    # ---------------------------------

    try:
        # --- ¡MODIFICADO! ---
        transcripcion, subtitulos_obj = leer_subtitulos_locales(RUTA_SUBTITULOS_LOCAL)

        if not transcripcion or not subtitulos_obj:
            raise Exception("No se pudo obtener la transcripción o los subtítulos.")

        secciones_ai = obtener_clips_con_gemini(API_KEY, transcripcion)
        if not secciones_ai:
            raise Exception("Gemini no devolvió secciones válidas.")

        print("\n--- Secciones identificadas por Gemini ---")
        print(json.dumps(secciones_ai, indent=2, ensure_ascii=False))
        print("------------------------------------------\n")

        # --- ¡MODIFICADO! ---
        cortar_video_clips(RUTA_VIDEO_LOCAL, secciones_ai, subtitulos_obj)

    except Exception as e:
        print(f"El proceso falló: {e}", file=sys.stderr)
        sys.exit(1)

ModuleNotFoundError: No module named 'moviepy.editor'